# Proyecto final Machine Learning

Jorge Ruizvisfocri

Emilio Martinez

# Descripción del proyecto

El siguiente proyecto busca clasificar imágenes de Carcinomas Ductales Invasivos (IDC en inglés) extraidas de muestras de pacientes con cancer de mama.

La base de datos fue tomada de https://www.kaggle.com/datasets/paultimothymooney/breast-histopathology-images

Se propone utilizar una red neuronal convolucional para resolver el problema de clasificación.

# Paqueterías de trabajo

In [1]:
## Paquetes de ciencias de datos
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import math
import random

## Paquetes de lectura de imagenes
from pathlib import Path
import os
import glob

## Paquetes de imágenes
from skimage.io import imread
import cv2

# Datos de trabajo

In [2]:
# Directorio de imágenes
data_dir = Path("D://bases de datos//proyecto_ml_final//imagenes") ## Directorio de datos

Dado que todos los pacientes tienen muestras con tumores malignos y benignos, la aleatorización puede realizarse a nivel de imagen o a nivel de paciente.

Bajo el supuesto de que podría existir correlación de algún tipo entre las imágenes pertenecientes a un mismo paciente, creemos que sería interesante aleatorizar a nivel de paciente, para introducirle a la red la información de personas que jamás ha visto.

In [3]:
### Aleatorización a nivel de paciente
#### Obtenemos la lista de pacientes
px = [f for f in data_dir.iterdir() if f.is_dir()]

In [4]:
### Obtenemos el número equivalente al porcentaje deseado
k = math.ceil( len(px) * 80 // 100)

In [18]:
### Hacemos la muestra de pacientes para entrenamiento y prueba

In [5]:
### fijamos la semilla para tener reproductibilidad
random.seed(4352)

In [6]:
train_id = random.sample(px,k)

In [7]:
test_id = set(px) - set(train_id)

In [8]:
### cargamos files de entrenamiento
train_images_files = []
for p in train_id:
    img_lst = list(p.rglob("*.png"))
    train_images_files.extend(img_lst)


In [10]:
test_images_files = []
for p in test_id:
    img_lst = list(p.rglob("*.png"))
    test_images_files.extend(img_lst)

In [13]:
### Revisamos que las direcciones de las imágenes estén bien cargadas
print(
    f"Número de imagenes\n"
    "-----------------\n"
    f"Total: {len(train_images_files) + len(test_images_files) }\n"  # 277,524 tiles
    "-----------------\n"
    f"Entrenamiento: {len(train_images_files) }\n"
    "-----------------\n"
    f"Prueba: {len(test_images_files) }\n"
)

Número de imagenes
-----------------
Total: 277524
-----------------
Entrenamiento: 220389
-----------------
Prueba: 57135



## Preparamos las etiquetas

In [16]:
## y_entrenamiento
### Preparamos las etiquetas
y_train = []
for name in train_images_files:
    etiqueta = str(name)[-5]
    etiqueta_num = float(etiqueta)
    y_train.append(etiqueta_num)

In [15]:
y_test = []
for name in test_images_files:
    etiqueta = str(name)[-5]
    etiqueta_num = float(etiqueta)
    y_test.append(etiqueta_num)

In [18]:
len(y_test)

57135

In [20]:
### Revisamos números de positivos en conjuntos
print(
    f"Positivos\n"
    "-----------------\n"
    f"Total: {sum(y_train) + sum(y_test)}\n"  # 277,524 tiles
    "-----------------\n"
    f"Entrenamiento: {sum(y_train) }\n"
    "-----------------\n"
    f"Prueba: {sum(y_test) }\n"
)

Positivos
-----------------
Total: 78786.0
-----------------
Entrenamiento: 64466.0
-----------------
Prueba: 14320.0



## Preparamos las imágenes

In [21]:
### Cargamos de imágenes de entrenamiento en una lista
dataset_img_train = []
for img in train_images_files:
    img_array = cv2.imread(str(img))
    dataset_img_train.append(img_array)

In [22]:
### Aplanamos las imágenes
data_set_img_train_plano = []
for img in dataset_img_train:
    img_plano = img.astype('float32') / 255
    data_set_img_train_plano.append(img_plano)

In [23]:
## Revisamos que tengan la misma longitud para el conjunto de entrenamiento
leny_train = len(y_train)
lenset_train = len(data_set_img_train_plano)

if leny_train == lenset_train:
    print("El tamaño del vector de resultados y de las imágenes es el mismo en el conjunto de entrenamiento")
else:
    print("El tamaño del vector de resultados y las imágenes no coincide en el conjunto de entrenamiento")

El tamaño del vector de resultados y de las imágenes es el mismo en el conjunto de entrenamiento


In [24]:
### Cargamos imágenes de prueba en una lista
dataset_img_test = []
for img in test_images_files:
    img_array = cv2.imread(str(img))
    dataset_img_test.append(img_array)

In [25]:
### Aplanamos las imágenes
data_set_img_test_plano = []
for img in dataset_img_test:
    img_plano = img.astype('float32') / 255
    data_set_img_test_plano.append(img_plano)

In [27]:
## Revisamos que tengan la misma longitud para el conjunto de prueba
leny_test = len(y_test)
lenset_test = len(data_set_img_test_plano)

if leny_test == lenset_test:
    print("El tamaño del vector de resultados y de las imágenes es el mismo en el conjunto de prueba")
else:
    print("El tamaño del vector de resultados y las imágenes no coincide en el conjunto de prueba")

El tamaño del vector de resultados y de las imágenes es el mismo en el conjunto de prueba


# Modelo